## Auto Refreshable STS Credentials 

By default the temporary credentials generated by AWS STS use the `GetSessionToken` API and are limited to **one** hour. You can extend the duration of your session by using `RefreshableCredentials`, a `botocore` class which can automatically refresh the credentials to work with your long running applications **beyond** the one-hour timeframe.<br><br>
This example notebook shows how to setup autorefreshable credentials that can be used with long running applications within AWS.

**Note:** This notebook can be run either in account A or B.

### Imports 

In [ ]:
from botocore.credentials import RefreshableCredentials
from botocore.session import get_session
from boto3 import Session
import botocore
import logging
import boto3

#### Setup Logging

In [ ]:
logger = logging.getLogger('sagemaker')
logger.setLevel(logging.INFO)
logger.addHandler(logging.StreamHandler())

logger.info(f'[Using botocore version: {botocore.__version__}]')
logger.info(f'[Using boto3 version: {boto3.__version__}]')

### Essentials

In [ ]:
# Use ARN of the feature store access role created in account B below. 
# E.g., arn:aws:iam::<ACCOUNT B ID>:role/cross-account-assume-role
CROSS_ACCOUNT_ASSUME_ROLE = '<ARN OF CROSS ACCOUNT ROLE CREATED IN ACCOUNT B>'
REGION = boto3.Session().region_name

### Create Auto-Refreshable Session

In [ ]:
class AutoRefreshableSession:
    def __init__(self, cross_account_assume_role, region):
        self.cross_account_assume_role = cross_account_assume_role
        self.region = region
        self.long_running_session = None
        self.create_auto_refreshable_session()
        
    def _refresh(self):
        """
        Refresh tokens by calling `assume_role` again.
        """
        sts_client = boto3.client('sts', region_name=self.region)
        params = {
            'RoleArn': self.cross_account_assume_role,
            'RoleSessionName': 'RefreshableSession',
            'DurationSeconds': 3600,
        }
        credentials = sts_client.assume_role(**params).get('Credentials')
        metadata = {
            'access_key': credentials.get('AccessKeyId'),
            'secret_key': credentials.get('SecretAccessKey'),
            'token': credentials.get('SessionToken'),
            'expiry_time': credentials.get('Expiration').isoformat()
        }
        return metadata
    
    def create_auto_refreshable_session(self):
        """
        Create an auto refreshable session using the metadata obtained 
        from the STS provided temporary credentials.
        """
        session = get_session()
        session_credentials = RefreshableCredentials.create_from_metadata(metadata=self._refresh(), 
                                                                          refresh_using=self._refresh, 
                                                                          method='sts-assume-role')
        session._credentials = session_credentials
        session.set_config_variable('region', self.region)
        self.long_running_session = Session(botocore_session=session)
        
    def get_session(self):
        return self.long_running_session

### Create Service Client
Now, lets validate the autofreshable credentials by creating a SageMaker service client using the `autorefresh_session` we created in the previous cells and list some feature groups we created under this account. We can also create any other AWS service clients this manner. E.g., Lambda, Athena etc.

In [ ]:
autorefresh_session = AutoRefreshableSession(cross_account_assume_role=CROSS_ACCOUNT_ASSUME_ROLE, 
                                             region=REGION).get_session()

In [ ]:
sagemaker_client = autorefresh_session.client('sagemaker', region_name=REGION)

In [ ]:
sagemaker_client.list_feature_groups()

**Note:** Re-run the above cell after an hour+ to check if you can use the `sagemaker_client` to list the feature groups again without any issues. Since, you are using an auto-refreshable session, it ensures that the STS credentials are automatically renewed every hour keeping the sessions and clients **active** to enable long running applications.